### Load Dataset 

In [117]:
from torch.utils.data import Dataset,DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm.notebook import tqdm
from torchmetrics import Accuracy

In [13]:
class catdogdataset(Dataset):
    def __init__(self,data_dir,transform=None) :
        super().__init__()
        self.data=ImageFolder(data_dir,transform)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
      


In [14]:
data_dir="dataset"
data=catdogdataset(data_dir,transform=None)

In [15]:
len(data)

10028

In [65]:
image,label=data[9333]
print(label)
image.size

1


(500, 499)

In [66]:
data_dir="dataset/test_set"

In [67]:
target_to_class={v: k for k ,v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'cats', 1: 'dogs'}


In [68]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128)),

])

In [69]:
train_data_dir="dataset/test_set"
test_data_dir="dataset/training_set"

train_folder=catdogdataset(train_data_dir,transform=transform)
test_folder=catdogdataset(test_data_dir,transform=transform)

train_dataloader=DataLoader(train_folder,shuffle=True,batch_size=32)
test_dataloader=DataLoader(test_folder,shuffle=True,batch_size=32)

In [70]:
for images,labels in train_dataloader:
    break

In [72]:
labels

tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 0, 0, 0])

In [71]:
images

tensor([[[[0.2256, 0.2309, 0.2330,  ..., 0.3210, 0.3079, 0.2660],
          [0.2052, 0.2110, 0.2141,  ..., 0.3282, 0.3075, 0.2659],
          [0.1956, 0.2019, 0.2084,  ..., 0.3221, 0.2749, 0.1869],
          ...,
          [0.1935, 0.1859, 0.1973,  ..., 0.0782, 0.0379, 0.0307],
          [0.1820, 0.1960, 0.1985,  ..., 0.1153, 0.0714, 0.0482],
          [0.1860, 0.1771, 0.1925,  ..., 0.1124, 0.0776, 0.0346]],

         [[0.2391, 0.2444, 0.2464,  ..., 0.3401, 0.3253, 0.2838],
          [0.2147, 0.2204, 0.2236,  ..., 0.3514, 0.3322, 0.2912],
          [0.2048, 0.2110, 0.2175,  ..., 0.3428, 0.2907, 0.2030],
          ...,
          [0.1974, 0.1899, 0.2012,  ..., 0.0879, 0.0364, 0.0290],
          [0.1859, 0.1999, 0.2024,  ..., 0.1210, 0.0618, 0.0363],
          [0.1899, 0.1810, 0.1964,  ..., 0.1199, 0.0890, 0.0295]],

         [[0.1557, 0.1611, 0.1629,  ..., 0.2010, 0.2146, 0.1744],
          [0.1542, 0.1598, 0.1624,  ..., 0.1914, 0.1791, 0.1386],
          [0.1469, 0.1526, 0.1586,  ..., 0

### CNN Model

In [109]:
class model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )
        self.classifier=nn.Linear(64*64*16,num_classes)
    def forward(self,x):
        x=self.feature_extraction(x)
        x=self.classifier(x)
        return x    
    

In [110]:
model=model(num_classes=2)

In [111]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model(
  (feature_extraction): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ELU(alpha=1.0)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ELU(alpha=1.0)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Linear(in_features=65536, out_features=2, bias=True)
)

### Training Loop

In [115]:
train_losses, val_losses = [], []
num_epochs=10
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images,labels in tqdm(train_dataloader, desc='Training loop'):
        optimizer.zero_grad()
        output = model(images).squeeze()  # Squeeze to match the label's shape if needed
        loss = criterion(output, labels)  
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_dataloader.dataset)
    train_losses.append(train_loss)


    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}")
    


Training loop:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1/10 - Train loss: 0.4382036093639019


Training loop:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 2/10 - Train loss: 0.3229264483128334


Training loop:   0%|          | 0/64 [00:00<?, ?it/s]

...

Epoch 8/10 - Train loss: 0.024081244791277375


Training loop:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 9/10 - Train loss: 0.015196415400054274


Training loop:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 10/10 - Train loss: 0.00963363499673902


In [122]:

acc=Accuracy(task="binary")
model.eval()
with torch.no_grad():
    for images,labels in test_dataloader:
        output=model(images)
        preds = torch.argmax(output, dim=1)

        acc(preds,labels.view(-1))

accuracy=acc.compute()
print(f"Accuracy:{accuracy}")

Accuracy:0.657963752746582
